In [1]:
!pip install pulp

In [2]:
from pulp import *
import numpy as np
import pandas as pd

In [3]:
file_path = '/content/drive/MyDrive/dataset.xlsx'  # 修改這裡
product_list = pd.read_excel(file_path, sheet_name='skincare product list', usecols="A:G", nrows=33)

**數據預處理**

In [4]:
product_list['capacity_ml'] = product_list['capacity'].str.extract(r'(\d+)').astype(int)

In [8]:
# 使用前向填充方法填补缺失的skin type和applicable age数据
product_list['skin type'].fillna(method='ffill', inplace=True)
product_list['applicable age'].fillna(method='ffill', inplace=True)

In [10]:
# 如果 'skin type' 和 'applicable age' 列是以换行符分隔的字符串，我们需要将它们转换为列表
product_list['skin type'] = product_list['skin type'].apply(lambda x: x.split('\n') if pd.notnull(x) else [])
product_list['applicable age'] = product_list['applicable age'].apply(lambda x: x.split('\n') if pd.notnull(x) else [])

In [22]:
product_list

,product name,category,Decision Variable,capacity,skin type,applicable age,price (NTD),capacity_ml
0,ADVANCED GENIFIQUE SERUM,serum,X1,30ml,"[Combination Skin, Dry Skin, Oily Skin, Sensit...","[Under 30, 31-45, Over 46]",3100,30
1,ADVANCED GENIFIQUE SERUM,serum,X2,50ml,"[Combination Skin, Dry Skin, Oily Skin, Sensit...","[Under 30, 31-45, Over 46]",4380,50
2,ADVANCED GENIFIQUE SERUM,serum,X3,75ml,"[Combination Skin, Dry Skin, Oily Skin, Sensit...","[Under 30, 31-45, Over 46]",5600,75
3,ADVANCED GENIFIQUE SERUM,serum,X4,115ml,"[Combination Skin, Dry Skin, Oily Skin, Sensit...","[Under 30, 31-45, Over 46]",8350,115
4,CLARIFIQUE REFINING BRIGHTENING PRO-SOLUTION,serum,X5,30ml,[Dry Skin],"[Under 30, 31-45]",3780,30
5,CLARIFIQUE REFINING BRIGHTENING PRO-SOLUTION,serum,X6,50ml,[Dry Skin],"[Under 30, 31-45]",5180,50
6,CLARIFYING REFINING DOUBLE TREATMENT ESSENCE,serum,X7,150ml,"[Combination Skin, Oily Skin]","[Under 30, 31-45]",3950,150
7,CLARIFYING REFINING DOUBLE TREATMENT ESSENCE,serum,X8,250ml,"[Combination Skin, Oily Skin]","[Under 30, 31-45]",5280,250
8,GENIFIQUE SENSITIVE,serum,X9,20ml,[Sensitive Skin],"[Under 30, 31-45, Over 46]",3250,20
9,RML ULTRA TRIPLE SERUM P B50ML AS,serum,X10,50ml,[Combination Skin],[31-45],6150,50


In [23]:
product_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product name       32 non-null     object
 1   category           32 non-null     object
 2   Decision Variable  32 non-null     object
 3   capacity           32 non-null     object
 4   skin type          32 non-null     object
 5   applicable age     32 non-null     object
 6   price (NTD)        32 non-null     int64 
 7   capacity_ml        32 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 2.1+ KB


**定義決策變量**

In [13]:
decision_variables = {row['Decision Variable']: LpVariable(row['Decision Variable'], cat='Binary')
                      for _, row in product_list.iterrows()}

**設定目標函數**

In [14]:
problem = LpProblem("Maximize_Revenue", LpMaximize)
problem += lpSum([row['price (NTD)'] * decision_variables[row['Decision Variable']]
                  for _, row in product_list.iterrows()]), "Total_Revenue"

**添加約束條件**

1. 每個套裝必須包含Cleanser、Toner、Serum和Eye Care：

In [15]:
for category in ['Cleanser', 'Toner', 'Serum', 'Eye Care']:
    products_in_category = product_list[product_list['category'].str.contains(category, case=False, na=False)]
    problem += lpSum([decision_variables[row['Decision Variable']] for _, row in products_in_category.iterrows()]) >= 1, f"{category}_in_set"


2. 如果套裝中有Lotion，就不能有Cream：

In [16]:
lotion_variables = [decision_variables[row['Decision Variable']] for _, row in product_list.iterrows() if isinstance(row['category'], str) and 'lotion' in row['category'].lower()]
cream_variables = [decision_variables[row['Decision Variable']] for _, row in product_list.iterrows() if isinstance(row['category'], str) and 'cream' in row['category'].lower()]
problem += (lpSum(lotion_variables) + lpSum(cream_variables)) <= 1, "Lotion_or_Cream"


3. 為每種膚質和年齡區間設置的約束：

In [17]:
skin_types = ['Combination Skin', 'Dry Skin', 'Oily Skin', 'Sensitive Skin', 'Normal Skin']
age_ranges = ['Under 30', '31-45', 'Over 46']


# 遍歷所有膚質和年齡組合
for skin_type in skin_types:
    for age_range in age_ranges:
        # 為每個膚質和年齡創建約束
        subset = product_list[
            product_list['skin type'].apply(lambda x: skin_type in x) &
            product_list['applicable age'].apply(lambda x: age_range in x)
        ]
        # 確保至少選擇一個適合目前膚質和年齡組合的產品
        problem += lpSum(decision_variables[row['Decision Variable']] for index, row in subset.iterrows()) >= 1, f"Availability_{skin_type}_{age_range}"


4. 添加容量約束

In [18]:
# 遍歷所有膚質和年齡組合來添加容量約束
for skin_type in skin_types:
    for age_range in age_ranges:
        subset = product_list[
            product_list['skin type'].apply(lambda x: skin_type in x) &
            product_list['applicable age'].apply(lambda x: age_range in x)
        ]
        # 確保每個set的總容量不超過500ml
        problem += lpSum(decision_variables[row['Decision Variable']] * row['capacity_ml'] for index, row in subset.iterrows()) <= 500, f"Capacity_{skin_type}_{age_range}"


5. 添加預算約束

In [19]:
# 定義預算限制
budget_constraints = {'Under 30': 10000, '31-45': 15000, 'Over 46': 20000}

# 遍歷所有膚質和年齡組合來添加預算約束
for skin_type in skin_types:
    for age_range in age_ranges:
        subset = product_list[
            product_list['skin type'].apply(lambda x: skin_type in x) &
            product_list['applicable age'].apply(lambda x: age_range in x)
        ]
        # 為每個膚質和年齡組和添加預算約束
        problem += lpSum(decision_variables[row['Decision Variable']] * row['price (NTD)'] for index, row in subset.iterrows()) <= budget_constraints[age_range], f"Budget_{skin_type}_{age_range}"


**求解**

In [20]:
problem.solve()


1